This is a tutorial of prior grn inference process. (Modified from Dictys.)

RNA and ATAC dataset

In [8]:
import os,sys
sys.path.append(os.path.realpath(os.path.join(os.getcwd(), '..')))
import warnings
warnings.filterwarnings("ignore") 
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
from SpatialGRN.config import config
opt = config()
args = opt.parse_args(['--gpu', '0'])
args.dataset_path = '../Data/Mouse_Brain'
args

Namespace(clusters=0, dataset_path='../Data/Mouse_Brain', embed_dim=10, eval=False, flow='source_to_target', gpu=0, hops=3, hvgs=3000, infer_mode='simplex', latent_dim=30, log_file='../Log', log_steps=10, lr=0.0005, max_epoch=20, max_neighbors=8, n_hops=3, n_randomwalk=[5, 10, 15], out_file='../Output', project_name='SpatialGRN', q_randomwalk=[1, 1.5], seed=2024, spread=0.003, srt_resolution=2, tau=1, version='v_1.0', visualize=True, wegiht_decay=1e-06)

In [7]:
from SpatialGRN.utils import set_random_seed
set_random_seed(args.seed)

In [30]:
adata = sc.read_h5ad(os.path.join(args.dataset_path, 'adata_RNA.h5ad'))
adata

AnnData object with n_obs × n_vars = 9215 × 22914
    obs: 'nCount_Spatial', 'nFeature_Spatial', 'nCount_SCT', 'nFeature_SCT', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_peaks', 'nFeature_peaks', 'RNA_clusters', 'ATAC_clusters'
    var: 'name'
    obsm: 'X_pca', 'X_umap', 'spatial'

In [43]:
"Ahr" in adata.var_names

True

In [55]:
adata = sc.read_h5ad(os.path.join(args.dataset_path, 'adata_peaks_normalized.h5ad'))
adata

AnnData object with n_obs × n_vars = 9215 × 121068
    obs: 'nCount_Spatial', 'nFeature_Spatial', 'nCount_SCT', 'nFeature_SCT', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_peaks', 'nFeature_peaks', 'RNA_clusters', 'ATAC_clusters'
    var: 'count', 'percentile'
    uns: 'ATAC', 'ATAC_clusters_colors', 'umap'
    obsm: 'X_lsi', 'X_pca', 'X_umap', 'spatial'
    obsp: 'ATAC_connectivities', 'ATAC_distances'

要设计的目标格式

In [24]:
import pandas as pd

# 示例DataFrame，包含TF、目标基因和它们之间的距离
df = pd.DataFrame({
    'tf': ['TF1', 'TF2', 'TF1', 'TF3'],
    'gene': ['GeneA', 'GeneB', 'GeneC', 'GeneA'],
    'distance': [1000, 500, 1200, 300]
})

# 构建邻接表
adjacency_table = pd.DataFrame({
    'TF': df['tf'],
    'Target Gene': df['gene'],
    'Distance': df['distance']
})

# 打印结果
print(adjacency_table)

# 将邻接表保存为CSV文件，便于以后加载
adjacency_table.to_csv('tf_binding_adjacency_table.csv', index=False, header=False)


    TF Target Gene  Distance
0  TF1       GeneA      1000
1  TF2       GeneB       500
2  TF1       GeneC      1200
3  TF3       GeneA       300


In [52]:
import pandas as pd
motif = pd.read_csv('/home/zl/SpatialGRN/Data/sc/motifs.motif', sep='\t')
motif

,>dKhGCGTGh,AHR_HUMAN.H11MO.0.B,3.3775000000000004
0.262728374765856,0.1227600511842322,0.362726,0.251786
0.07633328991810645,0.08258130543118362,0.225933,0.615152
0.14450570038747923,0.28392173880411337,0.138154,0.433418
0.023935814057894068,0.016203821748029118,0.925328,0.034532
0.007919544273173793,0.953597675415874,0.017308,0.021174
...,...,...,...
0.0015159934114744613,0.0015159934114744613,0.995452,0.001516
0.0015159934114744613,0.9885735697562056,0.001516,0.008394
0.9857791994398987,0.002160848099852988,0.009039,0.003021
0.17734637177601936,0.46452165966725645,0.168748,0.189384


In [57]:
from Bio.Seq import Seq
import numpy as np
from Bio import motifs
results = []
for idx, peak_seq in enumerate(adata.var_names):
    sequence = Seq(peak_seq)
    for position, score in motif.pssm.search(sequence, threshold=0.0):
        results.append({'peak_index': idx, 'position': position, 'score': score})

AttributeError: 'DataFrame' object has no attribute 'pssm'